In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
mainURL = "https://www.w3schools.com/html/default.asp"

page = requests.get(mainURL)
htmlContent = page.content
soup = BeautifulSoup(htmlContent, 'html.parser')

urls = []
pages = soup.find('div', {'id': 'leftmenuinnerinner'}).findAll(href=True)
for slug in pages:
    urls.append('https://www.w3schools.com/html/' + slug.get('href'))

mainHeadings = []
subHeadingsH2 = []
subHeadingsH3 = []
links = []
imageURLs = []
facts = []
definitions = []
explanations = []
codes = []
listItems = []
tableData = []
data = {}


In [3]:
for url in urls:
    page = requests.get(url)
    htmlContent = page.content
    soup = BeautifulSoup(htmlContent, 'html.parser')

    subContent = soup.find('div', {"id": "main"})  # Page Sub Content
    if subContent is None:
        continue

    mainHeading = subContent.findAll('h1')  # Main Heading
    pageMainHeadings = []
    if mainHeading is not None:
        for head in mainHeading:
            pageMainHeadings.append(head.text)

    subHeadingH2 = subContent.findAll('h2')  # Sub Headings h2
    pageSubHeadingsH2 = []
    if subHeadingH2 is not None:
        for head in subHeadingH2:
            pageSubHeadingsH2.append(head.text)

    subHeadingH3 = subContent.findAll('h3')  # Sub Headings h3
    pageSubHeadingsH3 = []
    if subHeadingH3 is not None:
        for head in subHeadingH3:
            pageSubHeadingsH3.append(head.text)

    link = subContent.findAll('a')  # Page Links
    pageLinks = []
    if link is not None:
        for lin in link:
            if lin.get('href') is not None and not lin.get('href').startswith('http'):
                pageLinks.append('https://www.w3schools.com/html/' + lin.get('href'))
            else:
                pageLinks.append(lin.get('href'))

    images = subContent.findAll('img')  # Images
    pageImageURLs = []
    if images is not None:
        for img in images:
            pageImageURLs.append('https://www.w3schools.com/html/' + img.get('src'))

    paragraphs = subContent.findAll('p')  # Paragraphs
    pageFacts = []
    pageDefinitions = []
    pageExplanations = []
    if paragraphs is not None:
        for para in paragraphs:
            slug = para.text.replace('\n', '')
            if 10 <= len(slug) <= 79:
                pageFacts.append(slug)
            elif 80 <= len(slug) <= 119:
                pageDefinitions.append(slug)
            elif 120 <= len(slug):
                pageExplanations.append(slug)

    codeContent = subContent.findAll('div', {"class": "w3-code"})  # Codes and Outputs
    pageCodes = []
    if codeContent is not None:
        for slug in codeContent:
            pageCodes.append(slug.text.replace('\xa0', ''))

    tables = subContent.findAll('table')  # Tables
    pageTableData = []
    if tables is not None:
        for tb in tables:
            table = ""
            rowData = []
            tableRows = tb.findAll('tr')
            for row in tableRows:
                rowData.append(row.text[1:-1].replace('\n', '|||'))
            for row in rowData:
                table = table + str(row) + '&&&'
            pageTableData.append(table[:-3])

    allListUL = subContent.findAll('ul')  # List Items
    allListOL = subContent.findAll('ol')
    listUL = ""
    listOL = ""
    listItem = ""
    pageListItems = []
    if len(allListUL) != 0 or len(allListOL) != 0:
        if len(allListUL) != 0:
            for lis in allListUL:
                listUL = listUL + str(lis.text[1:]).replace('\n', '|||').replace('\t\t', '')
        if len(allListOL) != 0:
            for lis in allListOL:
                listOL = listOL + str(lis.text[1:]).replace('\n', '|||').replace('\t\t', '')
        if len(allListUL) != 0 and len(allListOL) == 0:
            listItem = listItem + listUL
        elif len(allListUL) == 0 and len(allListOL) != 0:
            listItem = listItem + listOL
        else:
            listItem = listItem + listUL + '&&&' + listOL
        pageListItems.append(listItem)


    mainHeadings.append(pageMainHeadings)
    subHeadingsH2.append(pageSubHeadingsH2)
    subHeadingsH3.append(pageSubHeadingsH3)
    links.append(pageLinks)
    imageURLs.append(pageImageURLs)
    facts.append(pageFacts)
    definitions.append(pageDefinitions)
    explanations.append(pageExplanations)
    codes.append(pageCodes)
    tableData.append(pageTableData)
    listItems.append(pageListItems)

In [4]:
data = {'main_headings': mainHeadings, 'sub_headings_h2': subHeadingsH2, 'sub_headings_h3': subHeadingsH3,
        'links': links, 'image_urls': imageURLs, 'facts': facts, 'definitions': definitions,
        'explanations': explanations, 'codes': codes, 'table_data': tableData, 'list_items': listItems}

df = pd.DataFrame(data)
df.to_csv('W3Schools_HTML.csv')

In [5]:
a=pd.read_csv("W3Schools_HTML.csv")
a

,Unnamed: 0,main_headings,sub_headings_h2,sub_headings_h3,links,image_urls,facts,definitions,explanations,codes,table_data,list_items
0,0,['HTML Tutorial'],"['Easy Learning with HTML ""Try it Yourself""', ...",['Example'],['https://www.w3schools.com/html//default.asp'...,[],['HTML is the standard markup language for Web...,"['With our ""Try it Yourself"" editor, you can e...","['In this HTML tutorial, you will find more th...",['\n<!DOCTYPE html>\n<html><head><title>Page T...,[],[]
1,1,['HTML Introduction'],"['What is HTML?', 'A Simple HTML Document', 'W...","['Example', 'Example Explained']","['https://www.w3schools.com/html/default.asp',...",['https://www.w3schools.com/html/img_chrome.png'],['HTML is the standard markup language for cre...,"['The purpose of a web browser (Chrome, Edge, ...",['Note: Some HTML elements have no content (li...,['\n <!DOCTYPE html>\n<html><head><title>Page ...,['Start tag|||Element content|||End tag&&&<h1>...,['HTML stands for Hyper Text Markup Language||...
2,2,['HTML Editors'],"['Learn HTML Using Notepad or TextEdit', 'Step...",['Example'],['https://www.w3schools.com/html/html_intro.as...,['https://www.w3schools.com/html/img_notepad.p...,['A simple text editor is all you need to lear...,"['However, for learning HTML we recommend a si...",['Also change some preferences to get the appl...,['\n<!DOCTYPE html><html><body><h1>My First He...,[],[]
3,3,['HTML Basic Examples'],"['HTML Documents', 'The <!DOCTYPE> Declaration...","['Example', 'Example', 'Example', 'Example', '...",['https://www.w3schools.com/html/html_editors....,[],['In this chapter we will show some basic HTML...,['All HTML documents must start with a documen...,"['Right-click in an HTML page and select ""View...",['\n<!DOCTYPE html><html><body><h1>My First He...,[],[]
4,4,['HTML Elements'],"['HTML Elements', 'Nested HTML Elements', 'Nev...","['Example', 'Example Explained', 'Example', 'E...",['https://www.w3schools.com/html/html_basic.as...,[],"['An HTML element is defined by a start tag, s...",['HTML elements can be nested (this means that...,['Note: Some HTML elements have no content (li...,['\n<!DOCTYPE html>\n<html><body>\n <h1>My Fir...,['Start tag|||Element content|||End tag&&&<h1>...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
79,79,['HTML Quiz'],"['The Test', 'Count Your Score', 'Start the Qu...",[],['https://www.w3schools.com/html/html_examples...,[],"[""You can test your HTML skills with W3Schools...","[""The test is not official, it's just a nice w...",['You will get 1 point foreach correct answer....,[],[],[]
80,80,['HTML Exercises'],"['Exercises', 'Count Your Score', 'Start HTML ...",[],['https://www.w3schools.com/html/html_quiz.asp...,[],"[""You can test your HTML skills with W3Schools...",['We have gathered a variety of HTML exercises...,['Try to solve an exercise by editing some cod...,[],[],[]
81,81,['W3Schools HTML Certificate'],"['Document Your Skills', 'Get a Certificate', ...",[],['https://www.w3schools.com/html/html_exercise...,['https://www.w3schools.com/html//cert/pic_htm...,['W3Schools offers an Online Certification Pro...,['The perfect solution for busy professionals ...,"['Knowledge is power, especially in the curren...",[],['w3schools CERTIFIED . 2021||||||\xa0\xa0||...,['Study for free at W3Schools.com|||Study at y...
82,82,"['You Have Learned HTML, Now What?']","['HTML Summary', ""Now You Know HTML, What's Ne...","['Advantages:', 'Things to Consider:', 'Things...",['https://www.w3schools.com/html/html_exam.asp...,[],['This tutorial has taught you how to use HTML...,"['For more information on HTML, please take a ...",['HTML is the universal markup language for th...,[],[],['Use an Internet Service Provider|||Host your...
